In [1]:
import torch
print(f"MPS 장치를 지원하도록 build가 되었는가? {torch.backends.mps.is_built()}")
print(f"MPS 장치가 사용 가능한가? {torch.backends.mps.is_available()}") 

MPS 장치를 지원하도록 build가 되었는가? True
MPS 장치가 사용 가능한가? True


In [38]:
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("pilot_exaone")

LangSmith 추적을 시작합니다.
[프로젝트명]
pilot_exaone


# 그래프 State 정의

In [40]:
from typing import TypedDict, List, Dict
from langchain_core.documents import Document

class GraphState(TypedDict):
    filepath: str  # 원본 파일 경로
    analyzed_files: List  # 분석 완료된 파일 목록
    metadata: List[Dict]  # parsing metadata (api, model, usage)
    elements_from_parser: List[Dict]  # 파싱된 요소


In [41]:
state = GraphState(filepath='/Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/test_docs')
state

{'filepath': '/Users/daeunbaek/nuebaek/BOAZ/BOAZ_ADV/Daeun/test_docs'}

# DB

In [42]:
from langchain_core.documents import Document
import pickle

def load_docs_from_dicts(path):
    with open(path, "rb") as f:
        doc_dicts = pickle.load(f)
    docs = [Document(page_content=d["page_content"], metadata=d["metadata"]) for d in doc_dicts]
    print(f"✅ 로드 완료: {len(docs)} docs")
    return docs

In [43]:
# 불러오기
final_docs = load_docs_from_dicts("final_docs_dict.pkl")

✅ 로드 완료: 816 docs


In [44]:
final_docs

[Document(metadata={'type': 'text', 'page': 1, 'total_pages': 22, 'source': 'Airway management in neonates and infants European Society of Anaesthesiology and Intensive Care and British Journal of Anaesthesia joint guidelines.pdf', 'id': 5, 'paper_title': 'Airway management in neonates and infants', 'author': 'Takashi Asai, Evelien Cools, Alexandria Cronin, Thomas Engelhardt, John Fiadjoe, Alexander Fuchs, Garcia-Marcinkiewicz, Chloe Heath, Mathias Johansen, Jost Kaufmann, Maren Kleine-Brueggeney, Pete G. Kovatsis, Peter Kranke, Andrea C. Lusardi, Clyde Matava, James Peyton, Thomas Riva, Carolina S. Romero, Britta von Ungern-Sternberg, Francis Veyckemans, Arash Afshari', 'key_words': ['airway management, difﬁcult airway, neonate, paediatric', 'anaesthesia, practice guidelines'], 'header': 'European Society of Anaesthesiology and Intensive Care and British Journal of Anaesthesia joint guidelines', 'summary': 'Nicola Disma et al. collaboratively outline airway management guidelines, endo

In [46]:
from pinecone import Pinecone, ServerlessSpec
import os
# ✅ 환경변수에서 API 키 불러오기
api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("❌ PINECONE_API_KEY 환경변수가 설정되지 않았습니다.")

# ✅ Pinecone 클라이언트 생성
pc = Pinecone(api_key=api_key)

# ✅ 인덱스 이름 및 설정
index_name = "quickstart"
dimension = 1536  # 예: OpenAI embedding 모델 "text-embedding-ada-002" 사용 시

# ✅ 인덱스 생성 (이미 존재하는 경우 생략)
if index_name not in [i.name for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
    print(f"✅ 인덱스 '{index_name}' 생성 완료!")
else:
    print(f"✅ 인덱스 '{index_name}' 이미 존재합니다.")

✅ 인덱스 'quickstart' 이미 존재합니다.


In [14]:
from openai import OpenAI
from uuid import uuid4
from pinecone import Pinecone
import os
from tqdm import tqdm

# ✅ API 키 불러오기
openai_api_key = os.environ["OPENAI_API_KEY"]
pinecone_api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecone_api_key)

client = OpenAI(api_key=openai_api_key)
index = pc.Index("quickstart") 

# ✅ 메타데이터 정리 함수
def clean_metadata(meta: dict) -> dict:
    cleaned = {}
    for k, v in meta.items():
        if v is None:
            continue
        if isinstance(v, (str, int, float, bool)):
            cleaned[k] = v
        elif isinstance(v, list) and all(isinstance(i, str) for i in v):
            cleaned[k] = v
        else:
            cleaned[k] = str(v)
    return cleaned

# ✅ 업로드 루프
batch_size = 50
vectors = []

for i, doc in enumerate(tqdm(final_docs)):
    try:
        # 💡 임베딩 생성
        response = client.embeddings.create(
            model="text-embedding-ada-002",
            input=doc.page_content
        )
        embedding = response.data[0].embedding

        metadata = clean_metadata(doc.metadata)

        vectors.append({
            "id": str(uuid4()),
            "values": embedding,
            "metadata": metadata
        })

        # 📨 배치 업로드
        if len(vectors) == batch_size:
            index.upsert(vectors)
            vectors = []

    except Exception as e:
        print(f"❌ 에러 발생 (i={i}): {e}")

# 🔁 남은 벡터 업로드
if vectors:
    index.upsert(vectors)
    print("✅ 남은 벡터 업로드 완료")

100%|██████████| 816/816 [20:54<00:00,  1.54s/it]  


✅ 남은 벡터 업로드 완료


### Langgraph

In [ ]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import Pinecone as LangchainPinecone
from langchain_openai import OpenAIEmbeddings
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage
from langchain_teddynote import logging
from typing import TypedDict, List, Dict
import os

# LangSmith 프로젝트 설정 (LangGraph 로그 추적)
logging.langsmith("pilot_exaone")

LangSmith 추적을 시작합니다.
[프로젝트명]
pilot_exaone


In [ ]:
# 그래프 상태 정의
class GraphState(TypedDict):
    question: str
    documents: List[Document]
    answer: str
    messages: List

vector db / embedding

In [ ]:
# Pinecone 설정
api_key = os.environ.get("PINECONE_API_KEY")
if not api_key:
    raise ValueError("❌ PINECONE_API_KEY 환경변수가 설정되지 않았습니다.")
pc = Pinecone(api_key=api_key)
index_name = "quickstart"
index = pc.Index(index_name)

# 임베딩 모델 (OpenAI - ada)
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# 벡터스토어 연결
vectorstore = LangchainPinecone.from_existing_index(
    index_name=index_name,
    embedding=embedding_model,
    text_key="summary"
)

In [ ]:
# Retriever + Reranker 구성
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker = CrossEncoderReranker(model=cross_encoder, top_n=5)
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 10})
retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever
)

# LLM 연결 (Ollama - Exaone)
llm = ChatOllama(model='exaone3.5:7.8b', temperature=0.5)

# 프롬프트 템플릿 (한국어 응답)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Context:
{context}

#Question:
{question}

#Answer:"""
)

In [60]:
# 문서 검색 노드
def retrieve_documents(state: GraphState):
    question = state["question"]
    docs = retriever.invoke(question)
    return {**state, "documents": docs}

# LLM 답변 생성 노드 (참고 문서 포함)
def generate_answer(state: GraphState):
    question = state["question"]
    docs = state["documents"]
    context = "\n\n".join([doc.page_content for doc in docs])
    formatted_prompt = prompt.format(context=context, question=question)
    response = llm.invoke(formatted_prompt)
    content = response.content if isinstance(response, AIMessage) else str(response)

    # 참고 문서 정리
    references = []
    for i, doc in enumerate(docs):
        metadata = doc.metadata if hasattr(doc, "metadata") else {}
        source = metadata.get("source", "출처 없음")
        page = metadata.get("page", "페이지 없음")
        references.append(f"[{i+1}] 출처: {source}, 페이지: {page}")

    ref_text = "\n".join(references)
    full_response = f"{content}\n\n📚 참고 문서:\n{ref_text}"

    messages = [("user", question), ("assistant", full_response)]
    return {
        **state,
        "answer": full_response,
        "messages": add_messages(state.get("messages", []), messages)
    }

In [ ]:
# ✅ LangGraph 구성
builder = StateGraph(GraphState)
builder.add_node("retrieve", retrieve_documents)
builder.add_node("generate", generate_answer)
builder.set_entry_point("retrieve")
builder.add_edge("retrieve", "generate")
builder.set_finish_point("generate")
graph = builder.compile()

# 예시 실행
if __name__ == "__main__":
    memory = MemorySaver()
    question = "What is the review about direct oral anticoagulants?"
    state = GraphState(question=question, documents=[], answer="", messages=[])
    result = graph.invoke(state)
    print("\n답변:\n", result["answer"])


답변:
 이 리뷰는 직접 경구 항응고제(direct oral anticoagulants, DOACs)가 정형외과 수술 환자에서 지역 마취와 함께 사용될 때의 안전성과 효과에 대해 다루고 있습니다. 특히 Apixaban, Rivaroxaban, Dabigatran 같은 약물들이 마취 유형에 따른 안전성 프로파일을 비교 분석하였으며, 신경축상 마취(neuraxial anesthesia) 하에서는 혈전색전증 위험이 약간 낮게 나타났지만, Rivaroxaban의 경우 데이터가 제한적이기 때문에 주의가 필요하다는 점을 강조하고 있습니다.

📚 참고 문서:
[1] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 1.0
[2] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 6.0
[3] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 12.0
[4] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 2.0
[5] 출처: 1-s2.0-S0952818016300204-main.pdf, 페이지: 5.0


In [62]:
# 예시 실행
if __name__ == "__main__":
    memory = MemorySaver()
    question = "What are the standard guidelines for pediatric anesthesia monitoring?"
    state = GraphState(question=question, documents=[], answer="", messages=[])
    result = graph.invoke(state)
    print("\n답변:\n", result["answer"])


답변:
 제시된 문맥에서는 소아 마취 모니터링을 위한 표준 지침에 대한 직접적인 정보가 제공되지 않습니다. 문맥은 주로 마취 관련 가이드라인의 개정과 특정 기술의 사용에 초점을 맞추고 있지만, 소아 마취 모니터링에 특화된 내용은 포함하고 있지 않습니다. 따라서 이 질문에 대해 정확히 답변 드리기 어렵습니다. 더 구체적인 정보를 위해서는 관련 소아 마취 분야의 최신 가이드라인이나 연구 자료를 참고하시는 것이 좋을 것 같습니다.

📚 참고 문서:
[1] 출처: 1-s2.0-S0952818024000333-main.pdf, 페이지: 2.0
[2] 출처: 2022 American Society of Anesthesiologists Practice Guidelines for Management of the Difficult Airway.pdf, 페이지: 3.0
[3] 출처: 1-s2.0-S0952818024001375-main.pdf, 페이지: 5.0
[4] 출처: 1-s2.0-S0952818024001375-main.pdf, 페이지: 8.0
[5] 출처: 1-s2.0-S0952818024001375-main.pdf, 페이지: 8.0
